# Python &rarr; OKL Sandbox

AST Grammar info can be found in the [ast docs](https://docs.python.org/3/library/ast.html#abstract-grammar)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from occa import okl

In [9]:
@okl.kernel
def add_vectors(a: [float],
                b: [float],
                ab: [float]):
    for i in okl.range(len(a)).tile(16):
      ab[i] = a[i] + b[i]

In [10]:
add_vectors.__name__

'add_vectors'

In [11]:
print(add_vectors.__okl_source__)

@kernel void add_vectors(...) {
  for (int i = 0; i < 10; ++i) {
    ab[i] = a[i] + b[i]
  }
}


In [25]:
import ast
import inspect

def get_func_node(func):
    return ast.parse(inspect.getsource(func)).body[0]

In [26]:
def a(arg1, arg2, *, arg3, arg4):
    pass

get_func_node(a).args

In [27]:
_26.defaults

[]